In [1]:
import time
import os
import mysql.connector as sql
from mysql.connector import Error
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
from openpyxl import Workbook, load_workbook

In [2]:
# Configuration
EXCEL_FILE = "Odisha_Engineering_College_incubators.xlsx"
MYDB_CONFIG = {
    "host": "88.150.227.117",
    "user": "nrktrn_web_b16",
    "password": "nrktrn#^%5456",
    "database": "nrkindex_trn_b16",
    "port": "3306",
    "connect_timeout": 300
}
PORTAL_ID = 11324
MEMBER_ID = 1410
PARENT_PORTAL_ID = 11335
CATEGORY = "Engineering College and incubators"
DATE = "2024-09-10"
URL = "https://www.google.com/search?q=engineering+colleges+in+West+Bengal&tbm=lcl#rlfi=hd:;si:;mv:[[26.7265368,89.0349821],[21.8652738,85.8764742]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2"



In [3]:
# Initialize the WebDriver
driver = webdriver.Chrome()
driver.get(URL)
time.sleep(1)

# Load or create Excel workbook
if os.path.exists(EXCEL_FILE):
    workbook = load_workbook(EXCEL_FILE)
    sheet = workbook.active
else:
    workbook = Workbook()
    sheet = workbook.active
    sheet.append(["Title", "Link", "Address", "Phone", "Category", "Email", "Member ID", "Parent Portal ID", "Date"])

# Connect to MySQL
def connect_to_mysql():
    try:
        mydb = sql.connect(**MYDB_CONFIG)
        return mydb, mydb.cursor()
    except Error as e:
        print(f"Error connecting to MySQL server: {e}")
        driver.quit()
        exit(1)

mydb, mycursor = connect_to_mysql()

def process_element(element):
    global mydb, mycursor  # Ensure using global variables
    try:
        element.click()
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.XPATH, '//h2[@data-attrid="title"]')))
        
        title = driver.find_element(By.XPATH, '//h2[@data-attrid="title"]').text
        
        try:
            link = driver.find_element(By.CSS_SELECTOR, "a.dHS6jb")
            url = link.get_attribute("href")
            if "google" in url:
                url = ""
        except NoSuchElementException:
            url = ""

        try:
            address = driver.find_element(By.XPATH, '//span[@class="LrzXr"]').text
        except NoSuchElementException:
            address = ""

        try:
            phone_number = driver.find_element(By.XPATH, "//span[contains(@aria-label,'Call')]").text
            phone = phone_number.replace(" ", "")
        except NoSuchElementException:
            phone = ""

        try:
            email = driver.find_element(By.XPATH, '//a[contains(@href, "mailto:")]').text
        except NoSuchElementException:
            email = ""

        try:
            category = driver.find_element(By.XPATH, '//span[@class="YhemCb"]').text
        except NoSuchElementException:
            category = ""

        # Print data to terminal for debugging
        print(f"Title: {title}")
        print(f"Link: {url}")
        print(f"Address: {address}")
        print(f"Phone: {phone}")
        print(f"Category: {category}")
        print(f"Email: {email}")
        print(f"Member ID: {MEMBER_ID}")
        print(f"Parent Portal ID: {PARENT_PORTAL_ID}")
        print(f"Date: {DATE}")
        print("-" * 40)

        # Append data to the Excel sheet
        sheet.append([title, url, address, phone, category, email, MEMBER_ID, PARENT_PORTAL_ID, DATE])

        # Insert data into MySQL
        sql_query = """
            INSERT INTO kf_vendor (VEND_TITL, PORTAL_ID, MEMBERID, PARENTPORTALID, VEND_CON_ADDR, vend_url, phone, VEND_SDATE, VEND_CATEGRY)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        sql_values = (title, PORTAL_ID, MEMBER_ID, PARENT_PORTAL_ID, address, url, phone, DATE, category)
        
        try:
            mycursor.execute(sql_query, sql_values)
            mydb.commit()
        except Error as e:
            print(f"Error executing SQL query: {e}")
            if e.errno == 2013:
                print("Lost connection. Reconnecting...")
                mydb, mycursor = connect_to_mysql()  # Reconnect and reassign cursor

    except (ElementClickInterceptedException, StaleElementReferenceException) as e:
        print(f"Exception occurred: {e}. Skipping to the next element.")

# Process each element
def process_elements():
    elements = driver.find_elements(By.CSS_SELECTOR, ".vwVdIc")
    for element in elements:
        process_element(element)

process_elements()

# Pagination
num_loops = 300
for i in range(2, num_loops):
    aria_label_value = f'Page {i}'
    try:
        link_element = driver.find_element(By.CSS_SELECTOR, f'a[aria-label="{aria_label_value}"]')
        link_element.click()
        time.sleep(10)
        process_elements()
    except NoSuchElementException:
        print(f"Page {i} not found. Exiting loop.")
        break

# Save the Excel file and close the browser
workbook.save(EXCEL_FILE)
driver.quit()

Title: JIS College of Engineering
Link: 
Address: 
Phone: 
Category: 
Email: 
Member ID: 1410
Parent Portal ID: 11335
Date: 2024-09-10
----------------------------------------
Title: JIS College of Engineering
Link: 
Address: Barrackpore - Kalyani Expy, Block A5, Block A, Kalyani, West Bengal 741235
Phone: 03325808640
Category: 
Email: 
Member ID: 1410
Parent Portal ID: 11335
Date: 2024-09-10
----------------------------------------
Title: Asansol Engineering College (AEC)
Link: 
Address: PX82+66Q, Vivekananda Sarani, Opp. Jubilee Petrol Pump, Kanyapur, Asansol, West Bengal 713305
Phone: 03412253057
Category: 
Email: 
Member ID: 1410
Parent Portal ID: 11335
Date: 2024-09-10
----------------------------------------
Title: Netaji Subhash Engineering College
Link: 
Address: Mauza Ranabhutia, Techno City, Garia, Ranabhutia, West Bengal 700152
Phone: 09831817307
Category: 
Email: 
Member ID: 1410
Parent Portal ID: 11335
Date: 2024-09-10
----------------------------------------
Title: 
Link: